# Figure Manipulation

What's easy, what's annoying, and how to work around it.

In [ ]:
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio

from xarray_plotly import overlay, update_animation_traces

pio.renderers.default = "notebook_connected"

# Sample data
df = px.data.gapminder()
df_2007 = df.query("year == 2007")
df_countries = df.query("country in ['United States', 'China', 'Germany', 'Brazil']")

---
# Easy: Single Plots

All standard manipulation methods work as expected.

In [ ]:
fig = px.scatter(df_2007, x="gdpPercap", y="lifeExp", color="continent", size="pop")
fig

In [ ]:
# Layout
fig.update_layout(title="GDP vs Life Expectancy", template="plotly_white")

# All traces
fig.update_traces(marker_opacity=0.7)

# Specific traces
fig.update_traces(marker_line_width=2, selector={"name": "Europe"})

# Axes
fig.update_xaxes(type="log", title="GDP per Capita")
fig.update_yaxes(range=[40, 90])

# Annotations and shapes
fig.add_hline(y=df_2007["lifeExp"].mean(), line_dash="dash", line_color="gray")

fig

---
# Easy: Faceted Plots

`update_traces`, `update_xaxes`, `update_yaxes` all work across facets.

In [ ]:
fig = px.line(df_countries, x="year", y="gdpPercap", color="country", facet_col="country")
fig

In [ ]:
# Update ALL traces across all facets
fig.update_traces(line_width=3)

# Update ALL x-axes
fig.update_xaxes(showgrid=False)

# Update ALL y-axes
fig.update_yaxes(showgrid=False, type="log")

# Clean up facet labels
fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))

fig

### Targeting specific facets

Use `row=` and `col=` (1-indexed) to target specific facets.

In [ ]:
fig = px.histogram(px.data.tips(), x="total_bill", facet_row="sex", facet_col="time")

# Target specific cell
fig.update_yaxes(title_text="Frequency", row=1, col=1)

# Target entire column
fig.update_xaxes(title_text="Bill ($)", col=2)

# Target entire row
fig.update_traces(marker_color="orange", row=2)

fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))
fig

### Reference lines on facets

`add_hline`/`add_vline` apply to all facets by default. Use `row=`/`col=` to target.

In [ ]:
fig = px.scatter(df_2007, x="gdpPercap", y="lifeExp", facet_col="continent", facet_col_wrap=3)
fig.update_xaxes(type="log")

# Applies to ALL facets
fig.add_hline(y=70, line_dash="dash", line_color="red")

# Specific facet only
fig.add_hline(y=50, line_dash="dot", line_color="blue", row=2, col=1)

fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))
fig

---
# Easy: Adding traces to faceted/animated figures

Use `overlay` to add traces. It handles facets and animation frames automatically.

In [ ]:
# Animated scatter
fig = px.scatter(
    df_countries,
    x="gdpPercap",
    y="lifeExp",
    color="country",
    animation_frame="year",
    log_x=True,
    range_y=[40, 85],
)

# Create a figure with reference marker
ref = go.Figure(
    go.Scatter(
        x=[10000],
        y=[75],
        mode="markers",
        marker={"size": 20, "symbol": "star", "color": "gold"},
        name="Target",
    )
)

# Overlay - trace appears in all animation frames
combined = overlay(fig, ref)
combined

In [ ]:
# Faceted plot
fig = px.scatter(df_2007, x="gdpPercap", y="lifeExp", facet_col="continent", facet_col_wrap=3)
fig.update_xaxes(type="log")

# Add reference to first facet (default axes x, y)
ref1 = go.Figure(
    go.Scatter(
        x=[5000],
        y=[70],
        mode="markers",
        marker={"size": 15, "symbol": "star", "color": "gold"},
        name="Target 1",
    )
)

# Add reference to second facet (axes x2, y2)
ref2 = go.Figure(
    go.Scatter(
        x=[20000],
        y=[80],
        mode="markers",
        marker={"size": 15, "symbol": "star", "color": "red"},
        name="Target 2",
        xaxis="x2",
        yaxis="y2",  # specify target facet
    )
)

combined = overlay(fig, ref1, ref2)
combined.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))
combined

---
# Annoying: Facet axis names

To target a specific facet with `add_shape`, `add_annotation`, or when adding traces via `overlay`, you need to know the axis name (`x2`, `y3`, etc.).

In [ ]:
fig = px.scatter(df_2007, x="gdpPercap", y="lifeExp", facet_col="continent", facet_col_wrap=3)

# Inspect axis names
layout_dict = fig.layout.to_plotly_json()
print("X axes:", sorted([k for k in layout_dict if k.startswith("xaxis")]))
print("Y axes:", sorted([k for k in layout_dict if k.startswith("yaxis")]))

In [ ]:
# Check which trace uses which axis
for i, trace in enumerate(fig.data):
    print(f"Trace {i} ({trace.name}): xaxis={trace.xaxis or 'x'}, yaxis={trace.yaxis or 'y'}")

**Tip:** For simple cases, use `add_hline`/`add_vline` with `row=`/`col=` instead of `add_shape` - it handles axis mapping internally.

---
# Annoying: Animation trace updates

**This is the main pain point.** `update_traces()` does NOT update animation frames.

In [ ]:
fig = px.line(df_countries, x="year", y="gdpPercap", color="country", animation_frame="country")
fig

In [ ]:
# This only affects the INITIAL view, not the animation frames!
fig.update_traces(line_width=5, line_dash="dot")

print(f"Base trace line_width: {fig.data[0].line.width}")
print(f"Frame 0 trace line_width: {fig.frames[0].data[0].line.width}")

In [ ]:
# When you play the animation, it reverts to the frame's original style
fig

### Solution: `update_animation_traces`

xarray-plotly provides this helper to update both base traces and animation frames:

In [ ]:
# update_animation_traces is imported from xarray_plotly
# It updates traces in both base figure and all animation frames

In [ ]:
fig = px.line(df_countries, x="year", y="gdpPercap", color="country", animation_frame="country")

update_animation_traces(fig, line_width=4, line_dash="dot")

print(f"Base trace line_width: {fig.data[0].line.width}")
print(f"Frame 0 trace line_width: {fig.frames[0].data[0].line.width}")

In [ ]:
fig

### Selective updates with selector

Use `selector` to target specific traces by name:

In [ ]:
fig = px.line(df_countries, x="year", y="gdpPercap", color="country", animation_frame="year")

# Update only one trace by name
update_animation_traces(fig, selector={"name": "Germany"}, line_width=5, line_dash="dot")

# Update multiple traces
update_animation_traces(fig, selector={"name": "China"}, line_color="red", line_width=3)

fig

### Works with facets + animation

In [ ]:
df_subset = df.query(
    "continent in ['Europe', 'Asia'] and country in ['Germany', 'France', 'China', 'Japan']"
)

fig = px.line(
    df_subset,
    x="year",
    y="gdpPercap",
    color="country",
    facet_col="continent",
    animation_frame="year",
)

update_animation_traces(fig, line_width=3)
fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))
fig

### What's affected

Anything on **traces** needs the workaround for animations:

| Property | Facets | Animation |
|----------|--------|-----------|
| `line_width` | ✅ `update_traces()` | ❌ needs `update_animation_traces()` |
| `line_dash` | ✅ `update_traces()` | ❌ needs `update_animation_traces()` |
| `line_color` | ✅ `update_traces()` | ❌ needs `update_animation_traces()` |
| `marker_size` | ✅ `update_traces()` | ❌ needs `update_animation_traces()` |
| `marker_symbol` | ✅ `update_traces()` | ❌ needs `update_animation_traces()` |
| `opacity` | ✅ `update_traces()` | ❌ needs `update_animation_traces()` |

**Layout properties** (`update_layout`, `update_xaxes`, `update_yaxes`) work fine for animations.

---
# Annoying: Animation speed

The API to change animation speed is deeply nested.

In [ ]:
fig = px.scatter(
    df,
    x="gdpPercap",
    y="lifeExp",
    color="continent",
    size="pop",
    animation_frame="year",
    log_x=True,
    range_y=[25, 90],
)

# This is... not intuitive
fig.layout.updatemenus[0].buttons[0].args[1]["frame"]["duration"] = 100  # faster
fig.layout.updatemenus[0].buttons[0].args[1]["transition"]["duration"] = 50

fig

### Workaround: Helper function

In [ ]:
def set_animation_speed(fig, frame_duration=500, transition_duration=300):
    """Set animation speed in milliseconds."""
    if fig.layout.updatemenus:
        fig.layout.updatemenus[0].buttons[0].args[1]["frame"]["duration"] = frame_duration
        fig.layout.updatemenus[0].buttons[0].args[1]["transition"]["duration"] = transition_duration
    return fig

In [ ]:
fig = px.scatter(
    df,
    x="gdpPercap",
    y="lifeExp",
    color="continent",
    animation_frame="year",
    log_x=True,
    range_y=[25, 90],
)

set_animation_speed(fig, frame_duration=200, transition_duration=100)
fig

---
# Annoying: Slider styling

Verbose but straightforward.

In [ ]:
fig = px.scatter(
    df,
    x="gdpPercap",
    y="lifeExp",
    color="continent",
    animation_frame="year",
    log_x=True,
    range_y=[25, 90],
)

fig.layout.sliders[0].currentvalue.prefix = "Year: "
fig.layout.sliders[0].currentvalue.font.size = 16
fig.layout.sliders[0].pad.t = 50  # padding from top

fig

### Hide slider or play button

In [ ]:
fig = px.scatter(
    df,
    x="gdpPercap",
    y="lifeExp",
    color="continent",
    animation_frame="year",
    log_x=True,
    range_y=[25, 90],
)

# Hide slider (keep play button)
fig.layout.sliders = []

# Or hide play button (keep slider):
# fig.layout.updatemenus = []

fig

---
# Summary

### Provided by xarray-plotly

```python
from xarray_plotly import overlay, update_animation_traces
```

### Local helper for animation speed

In [ ]:
def set_animation_speed(fig, frame_duration=500, transition_duration=300):
    """Set animation speed in milliseconds."""
    if fig.layout.updatemenus:
        fig.layout.updatemenus[0].buttons[0].args[1]["frame"]["duration"] = frame_duration
        fig.layout.updatemenus[0].buttons[0].args[1]["transition"]["duration"] = transition_duration
    return fig

### Quick reference

| Task | Facets | Animation | Solution |
|------|--------|-----------|----------|
| Update trace style | `update_traces()` | `update_animation_traces()` | Helper needed |
| Update axes | `update_xaxes()`/`update_yaxes()` | Same | ✅ Works |
| Update layout | `update_layout()` | Same | ✅ Works |
| Add reference line | `add_hline(row=, col=)` | `add_hline()` | ✅ Works |
| Add trace | `overlay()` | `overlay()` | ✅ Works |
| Add shape to specific facet | `add_shape(xref="x2")` | Same | Need axis name |
| Change animation speed | N/A | `set_animation_speed()` | Helper needed |
| Facet labels | `for_each_annotation()` | Same | ✅ Works |